# Import

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import subprocess
import RNA
from Bio import Entrez, SeqIO
import pandas as pd
from base64 import b64encode
import json
from urllib import request, parse
import requests

%matplotlib inline

# primer generation and alignment

In [2]:
# ---------- Настройки Entrez ----------
Entrez.email = 'your_email@example.com'

# ---------- Шаг 0: Загрузка последовательности из NCBI ----------
def fetch_gene_sequence(gene_name, database='nucleotide'):
    try:
        handle = Entrez.esearch(db=database, term=gene_name, retmax=1)
        record = Entrez.read(handle)
        handle.close()

        if not record['IdList']:
            raise ValueError(f"Ген '{gene_name}' не найден в базе данных {database}.")

        gene_id = record['IdList'][0]

        handle = Entrez.efetch(db=database, id=gene_id, rettype='fasta', retmode='text')
        record = SeqIO.read(handle, 'fasta')
        handle.close()

        print(f"Последовательность гена '{gene_name}' успешно загружена.")
        return str(record.seq)
    except Exception as e:
        raise RuntimeError(f'Ошибка при загрузке последовательности: {e}')

# ---------- Шаг 1: Генерация праймеров с помощью Primer3 ----------
def run_primer3(sequence):
    if len(sequence) < 50:
        raise ValueError('Длина последовательности слишком мала для генерации праймеров!')
    input_data = f"""
SEQUENCE_ID=example
SEQUENCE_TEMPLATE={sequence}
PRIMER_OPT_SIZE=20
PRIMER_MIN_SIZE=18
PRIMER_MAX_SIZE=25
PRIMER_PRODUCT_SIZE_RANGE=100-250
PRIMER_MIN_TM=55.0
PRIMER_OPT_TM=65.0
PRIMER_MAX_TM=70.0
PRIMER_MIN_GC=40
PRIMER_MAX_GC=60
PRIMER_NUM_RETURN=5
PRIMER_MAX_HAIRPIN_TH=20.0
PRIMER_MAX_SELF_ANY_TH=20.0
PRIMER_MAX_SELF_END_TH=20.0
SEQUENCE_INCLUDED_REGION=100,500
=
"""
    with open('primer3_input.txt', 'w') as f:
        f.write(input_data.strip())
    
    subprocess.run(['primer3_core', 'primer3_input.txt', '-output', 'primer3_output.txt'])
    with open('primer3_output.txt', 'r') as f:
        return f.read()

def extract_primer(primer_output, pair_index, side):
    key = f'PRIMER_{side}_{pair_index}_SEQUENCE='
    for line in primer_output.split('\n'):
        if line.startswith(key):
            return line.split('=')[-1]
    return None

def extract_primer_tm(primer_output, pair_index, side):
    key = f'PRIMER_{side}_{pair_index}_TM='
    for line in primer_output.split('\n'):
        if line.startswith(key):
            return float(line.split('=')[-1])
    return None

def extract_product_size(primer_output, pair_index):
    key = f'PRIMER_PAIR_{pair_index}_PRODUCT_SIZE='
    for line in primer_output.split('\n'):
        if line.startswith(key):
            return int(line.split('=')[-1])
    return None

# ---------- Шаг 2: Проверка вторичных структур с помощью ViennaRNA ----------
def analyze_with_vienna(sequence):
    structure, mfe = RNA.fold(sequence)
    has_hairpin = '(' in structure and ')' in structure
    return {
        'structure': structure,
        'mfe': mfe,
        'has_hairpin': has_hairpin
    }

# ---------- Полный анализ ----------
def analyze_primer_for_gene(gene_name):
    print(f"Шаг 0: Загрузка последовательности для гена '{gene_name}'...")
    sequence = fetch_gene_sequence(gene_name)

    print('\nШаг 1: Генерация праймеров с помощью Primer3...')
    primers = run_primer3(sequence)

    print('\nШаг 2: Проверка праймеров с помощью ViennaRNA...')
    primer_issues = analyze_primers_with_criteria(primers)

    results = {
        'GeneSequence': sequence,
        'Primer3': primers,
        'PrimerIssues': primer_issues,
    }

    display_results(results)
    return results

# ---------- Переработанный вывод ----------
def format_primer_issues(primer_issues):
    formatted_issues = []
    for issue in primer_issues:
        pair = issue['Pair']
        problems = issue['Issues']
        if problems:
            formatted_issues.append(f"Пара {pair + 1}:\n  - " + '\n  - '.join(problems))
    return '\n\n'.join(formatted_issues)

def display_results(results):
    print('=== Итоговый анализ праймеров ===\n')
    print(f"Длина последовательности гена: {len(results['GeneSequence'])} bp\n")

    primer_issues = results['PrimerIssues']
    if primer_issues:
        print('Проблемы, найденные при анализе праймеров:\n')
        print(format_primer_issues(primer_issues))
    else:
        print('Все праймеры соответствуют критериям.')

In [22]:
gene_name = "MT-ND1"
results = analyze_primer_for_gene(gene_name)

Шаг 0: Загрузка последовательности для гена 'MT-ND1'...
Последовательность гена 'MT-ND1' успешно загружена.

Шаг 1: Генерация праймеров с помощью Primer3...

Шаг 2: Проверка праймеров с помощью ViennaRNA...
=== Итоговый анализ праймеров ===

Длина последовательности гена: 357798456 bp

Проблемы, найденные при анализе праймеров:

Пара 1:
  - Шпилька в левом праймере с ΔG = -2.0 ккал/моль
  - Шпилька в правом праймере с ΔG = -1.899999976158142 ккал/моль

Пара 2:
  - Шпилька в левом праймере с ΔG = -2.0 ккал/моль
  - Шпилька в правом праймере с ΔG = -1.899999976158142 ккал/моль

Пара 3:
  - Температура отжига левого праймера (62.625°C) вне допустимого диапазона
  - Шпилька в левом праймере с ΔG = -2.0 ккал/моль
  - Шпилька в правом праймере с ΔG = -1.899999976158142 ккал/моль

Пара 4:
  - Температура отжига правого праймера (62.016°C) вне допустимого диапазона
  - Шпилька в левом праймере с ΔG = -2.0 ккал/моль
  - Шпилька в правом праймере с ΔG = -1.899999976158142 ккал/моль

Пара 5:
  - 

# analysis using IDT API

In [2]:
class OligoAnalyzerAPI:
    """
    Класс для взаимодействия с OligoAnalyzer API.
    """

    TOKEN_URL = "https://eu.idtdna.com/Identityserver/connect/token"
    SELF_DIMER_URL = "https://eu.idtdna.com/restapi/v1/OligoAnalyzer/SelfDimer"
    HETERO_DIMER_URL = "https://eu.idtdna.com/restapi/v1/OligoAnalyzer/HeteroDimer"

    def __init__(self, client_id, client_secret, username, password):
        self.client_id = client_id
        self.client_secret = client_secret
        self.username = username
        self.password = password
        self.api_token = None

    def get_access_token(self):
        """
        Получает токен доступа для API OligoAnalyzer.
        """
        try:
            authorization_string = b64encode(
                bytes(f"{self.client_id}:{self.client_secret}", "utf-8")
            ).decode()
            headers = {
                "Content-Type": "application/x-www-form-urlencoded",
                "Authorization": f"Basic {authorization_string}",
            }
            data = parse.urlencode({
                "grant_type": "password",
                "scope": "test",
                "username": self.username,
                "password": self.password,
            }).encode()

            req = request.Request(self.TOKEN_URL, data=data, headers=headers, method="POST")
            response = request.urlopen(req)
            body = json.loads(response.read().decode())

            if response.status != 200:
                raise RuntimeError(f"Ошибка получения токена: {response.status}")

            self.api_token = body["access_token"]
            print("Токен успешно получен!")
        except Exception as e:
            print(f"Ошибка при получении токена: {e}")
            self.api_token = None

    def analyze_self_dimer(self, primer):
        """
        Анализ гомодимеров (Self-Dimer).
        """
        if not self.api_token:
            print("Нет токена доступа!")
            return None

        headers = {"Authorization": f"Bearer {self.api_token}"}
        params = {"primary": primer}

        try:
            response = requests.post(self.SELF_DIMER_URL, headers=headers, params=params)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"Ошибка при анализе Self-Dimer: {e}")
            return None

    def analyze_hetero_dimer(self, forward_primer, reverse_primer):
        """
        Анализ гетеродимеров (Hetero-Dimer).
        """
        if not self.api_token:
            print("Нет токена доступа!")
            return None

        headers = {"Authorization": f"Bearer {self.api_token}"}
        params = {"primary": forward_primer, "secondary": reverse_primer}

        try:
            response = requests.post(self.HETERO_DIMER_URL, headers=headers, params=params)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"Ошибка при анализе Hetero-Dimer: {e}")
            return None


class DimerVisualizer:
    """
    Класс для визуализации димеров.
    """

    @staticmethod
    def visualize(sequence, bonds, delta_g, base_pairs, start_position=0,
                  top_padding=0, bond_padding=0, bottom_padding=0,
                  title="Dimer Visualization", secondary_sequence=None):
        """
        Текстовая визуализация димера.
        """
        if secondary_sequence is None:
            secondary_sequence = sequence[::-1]

        top_line = " " * (top_padding + start_position) + f"5'  {sequence}  3'"
        connection_line = " " * (4 + bond_padding + start_position)
        for bond in bonds:
            connection_line += "|" if bond == 2 else "." if bond == 1 else " "
        bottom_line = " " * (bottom_padding + start_position) + f"3'  {secondary_sequence}  5'"

        max_length = max(len(top_line), len(connection_line), len(bottom_line))
        print(f"\n{title}\nDelta G: {delta_g} ккал/моль | Base Pairs: {base_pairs}")
        print(top_line.ljust(max_length))
        print(connection_line.ljust(max_length))
        print(bottom_line.ljust(max_length))
        print("-" * max_length)

In [3]:
# Пример использования
client_id = "21"
client_secret = "6d31fff8-d7cf-4ee4-98d8-d1ba69940f65"
username = "takul"
password = "ULAsFp5J6zjS24E"

api = OligoAnalyzerAPI(client_id, client_secret, username, password)
api.get_access_token()

if api.api_token:
    forward_primer = "ACCTCCTCCTCTTCTGGTTGGT"
    reverse_primer = "TTCCGCTCCTGGATGTCCCTTG"

    forward_dimer = api.analyze_self_dimer(forward_primer)
    reverse_dimer = api.analyze_self_dimer(reverse_primer)
    hetero_dimer = api.analyze_hetero_dimer(forward_primer, reverse_primer)

    visualizer = DimerVisualizer()

    if forward_dimer:
        for result in forward_dimer:
            visualizer.visualize(
                forward_primer,
                result['Bonds'],
                result['DeltaG'],
                result['BasePairs'],
                result.get('StartPosition', 0),
                result.get('TopLinePadding', 0),
                result.get('BondLinePadding', 0),
                result.get('BottomLinePadding', 0),
                title="Forward Self-Dimer"
            )

    if reverse_dimer:
        for result in reverse_dimer:
            visualizer.visualize(
                reverse_primer,
                result['Bonds'],
                result['DeltaG'],
                result['BasePairs'],
                result.get('StartPosition', 0),
                result.get('TopLinePadding', 0),
                result.get('BondLinePadding', 0),
                result.get('BottomLinePadding', 0),
                title="Reverse Self-Dimer"
            )

    if hetero_dimer:
        for result in hetero_dimer:
            visualizer.visualize(
                forward_primer,
                result['Bonds'],
                result['DeltaG'],
                result['BasePairs'],
                result.get('StartPosition', 0),
                result.get('TopLinePadding', 0),
                result.get('BondLinePadding', 0),
                result.get('BottomLinePadding', 0),
                title="Hetero-Dimer",
                secondary_sequence=reverse_primer
            )
else:
    print("Анализ не выполнен.")

Токен успешно получен!

Forward Self-Dimer
Delta G: -4.41 ккал/моль | Base Pairs: 3
    5'  ACCTCCTCCTCTTCTGGTTGGT  3'
        |||            ...        
3'  TGGTTGGTCTTCTCCTCCTCCA  5'    
----------------------------------

Forward Self-Dimer
Delta G: -4.41 ккал/моль | Base Pairs: 3
5'  ACCTCCTCCTCTTCTGGTTGGT  3'
    |||  .          .  ...    
3'  TGGTTGGTCTTCTCCTCCTCCA  5'
------------------------------

Forward Self-Dimer
Delta G: -3.07 ккал/моль | Base Pairs: 2
 5'  ACCTCCTCCTCTTCTGGTTGGT  3'
      .  ||         ..  .      
3'  TGGTTGGTCTTCTCCTCCTCCA  5' 
-------------------------------

Forward Self-Dimer
Delta G: -3.07 ккал/моль | Base Pairs: 2
5'  ACCTCCTCCTCTTCTGGTTGGT  3'  
        .  ||      ..  .        
  3'  TGGTTGGTCTTCTCCTCCTCCA  5'
--------------------------------

Forward Self-Dimer
Delta G: -3.07 ккал/моль | Base Pairs: 2
5'  ACCTCCTCCTCTTCTGGTTGGT  3'   
        ||  .       .  ..        
   3'  TGGTTGGTCTTCTCCTCCTCCA  5'
---------------------------------

Forward Sel